In [ ]:
import sys
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from functools import partial

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils.dataset_tokenization import process_data

In [18]:
# model_path = "../../self-corrective-llama_untrained"
model_name = "MathBite/self_corrective_llama_3.1_8B_model_untrained"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# print(model)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    # Specify the task type
    task_type=TaskType.CAUSAL_LM,
    
    # LoRA-specific parameters
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    
    # Specify which modules to apply LoRA to.
    target_modules=[
        "q_proj", "k_proj", "v_proj", 
        "o_proj", "lm_head" 
    ],
    
    # Specify which modules to make fully trainable (not LoRA-fied).
    modules_to_save=["hallucination_detector"],
)

peft_model = get_peft_model(model, peft_config)

print("--- PEFT Model Trainable Parameters ---")
peft_model.print_trainable_parameters()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
--- PEFT Model Trainable Parameters ---
trainable params: 5,494,833 || all params: 1,241,317,426 || trainable%: 0.4427


/Users/kyrylo/miniforge3/envs/ai/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/kyrylo/miniforge3/envs/ai/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [19]:
data_path = "../../dataset/train.json"
dataset = datasets.load_dataset("json", data_files=data_path)

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'incorrect_response', 'errors', 'correct_response', 'additional_info'],
        num_rows: 46722
    })
})

In [21]:
SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
DELETION_TOKEN_IDS = set(tokenizer.convert_tokens_to_ids(DELETION_MARKERS))

mapper = partial(
    process_data,
    tokenizer=tokenizer,
    special_instruction=SPECIAL_INSTRUCTION,
    insertion_marker=INSERTION_MARKER,
    deletion_token_ids=DELETION_TOKEN_IDS
)

In [22]:
tokenized_dataset = dataset.map(mapper, batched=False)
tokenized_dataset = tokenized_dataset["train"]
columns_to_remove = [
    "input", "correct_response", "incorrect_response", 
    "additional_info", "errors"
]

tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)


In [23]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 46722
})

In [24]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
print(split_dataset)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
        num_rows: 42049
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
        num_rows: 4673
    })
})


In [25]:
output_dir = "../../dataset/training"
split_dataset.save_to_disk(output_dir)

Saving the dataset (0/1 shards):   0%|          | 0/42049 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4673 [00:00<?, ? examples/s]

In [ ]:
tmp = dataset.load_from_disk("../../dataset/training")
print(tmp)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
        num_rows: 42049
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
        num_rows: 4673
    })
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 4673
})


'train'

In [ ]:
from transformers import TrainingArguments
from src.trainer import SelfCorrectionTrainer, SelfCorrectionDataCollator

# These arguments control every aspect of the training run.
training_args = TrainingArguments(
    # --- Core Parameters ---
    output_dir="self-corrective-llama-finetuned", # Where to save the model
    num_train_epochs=1,                          # A good starting point for LoRA
    
    # --- Batching and Memory ---
    # Reduce batch size to prevent out-of-memory errors. Increase as your GPU allows.
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # Use gradient accumulation to simulate a larger batch size
    gradient_accumulation_steps=8,
    
    # --- Optimizer and Learning Rate ---
    optim="paged_adamw_8bit", # Memory-efficient optimizer
    learning_rate=2e-4,     # A common, effective learning rate for LoRA
    weight_decay=0.01,
    lr_scheduler_type="cosine", # A popular learning rate scheduler
    
    # --- Performance and Precision ---
    bf16=True, # Use bfloat16 for faster training on compatible GPUs (Ampere or newer)
    # fp16=True, # Use fp16 on older GPUs if bf16 is not available
    
    # --- Logging and Saving ---
    logging_dir="logs",
    logging_strategy="steps",
    logging_steps=10,
    # evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    report_to="wandb",
)

data_collator = SelfCorrectionDataCollator(tokenizer=tokenizer)

trainer = SelfCorrectionTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    alpha=0.7,
)

print("Trainer setup is complete and ready for training.")

/Users/kyrylo/Programming/Uni/Masters/Self-Corrective LLM/src/trainer.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SelfCorrectionTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Trainer setup is complete and ready for training.


In [12]:
trainer.train()

wandb: Currently logged in as: kyrylldekanenko (kyrylldekanenko-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [ ]:
# import torch

# # Get a single raw sample from the dataset (values are lists)
# data_sample = train_dataset[0]

# # --- Manually prepare the sample for the model ---

# # Also remove labels, since our custom trainer handles them separately
# data_sample.pop("labels", None)
# data_sample.pop("hallucination_labels", None)

# # 2. Convert lists to tensors, add a batch dimension (the outer []), and move to the correct device
# # The model expects a batch, even if it's just a batch of 1.
# input_ids_tensor = torch.tensor([data_sample["input_ids"]]).to(model.device)
# attention_mask_tensor = torch.tensor([data_sample["attention_mask"]]).to(model.device)

# model_inputs = {
#     "input_ids": input_ids_tensor,
#     "attention_mask": attention_mask_tensor
# }

# print("--- Model Inputs (Tensors) ---")
# print(model_inputs)

# # 3. Pass the prepared tensor inputs to the model
# # The ** operator unpacks the dictionary into keyword arguments
# with torch.no_grad(): # Use no_grad for inference to save memory
#     res = model(**model_inputs)

# print("\n--- Model Outputs ---")
# print(res)